In [1]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from dateutil.rrule import rrule, MONTHLY
from pprint import pprint as pp
import json
import os
import shutil
import numpy as np
from collections import defaultdict

In [2]:
from modules.aux_functions import *
from modules.global_vars import *
from modules.reservation_management import get_reservation_code
from modules.table_manager import manage_table, write_table_data, get_table_data

In [3]:
np.random.seed(583)

In [4]:
table_name = '4_double-superior_lodge_weekend_ls'
core_tables_path = 'Tablas_actu/Tablas_hotel/'
price_days_tables_path = 'Tablas_actu/Dias_por_precio/'

columns = ['p', 'p01', 'p02', 'outcome_prob', 'm_clients', 'm_demand', 'n_days', 't_clients']
index = ['p', 'p01', 'p02']

hotel1 = "hotel1"
hotel2 = "hotel2"

updated_comp_prices = {
    "0": {
        "hotel1": 110,
        "hotel2": 130
    },
    "1": {
        "hotel1": 110,
        "hotel2": 130
    },
    "2": {
        "hotel1": 110,
        "hotel2": 120
    },
    "3": {
        "hotel1": 100,
        "hotel2": 110
    },
    "4": {
        "hotel1": 90,
        "hotel2": 100
    }
}

In [5]:
new_reservation = {
    TIMESTAMP: datetime.now(),
    DATE_FROM: datetime.now() + timedelta(days=2),
    DATE_TO: datetime.now() + timedelta(days=3),
    ROOM: "doble superior",
    BOARD: "Solo Alojamiento",
    PRICE: 120,
    DAYS_IN_ADVANCE: 2
}

new_cm_update = {
    "0": {
        "hotel1": 110,
        "hotel2": 120
    },
    "1": {
        "hotel1": 110,
        "hotel2": 120
    },
    "2": {
        "hotel1": 110,
        "hotel2": 110
    },
    "3": {
        "hotel1": 100,
        "hotel2": 110
    },
    "4": {
        "hotel1": 90,
        "hotel2": 100
    }
}

In [6]:
def event_new_reservation(new_reservation):
    #Local variables
    reservation_code = get_reservation_code(new_reservation)
    if reservation_code == "ERROR":
        pp("Error") 
        
    room_price = new_reservation[PRICE]
    loc_days_in_advance = new_reservation[DAYS_IN_ADVANCE]
    comp_prices = [loc_price for key, loc_price in updated_comp_prices[str(loc_days_in_advance)].items()]
    table_path = core_tables_path + reservation_code + ".csv"
    
    if os.path.isfile(table_path):
        df = pd.read_csv(table_path, index_col=index)
    else:
        df = pd.DataFrame(columns=columns)
        df.set_index(index, inplace=True)
        
    loc_index = [room_price] + comp_prices
    loc_index_tup = tuple(loc_index)
    
    if loc_index_tup in df.index:
        df.loc[loc_index_tup, "t_clients"] += 1
    else:
        df.loc[loc_index_tup] = [0, 0, 0, 0, 1]
    
    pp(df)
    df.to_csv(table_path, sep=',', encoding='utf-8')

In [8]:
event_new_reservation(new_reservation)

[110, 120]
             outcome_prob  m_clients  m_demand  n_days  t_clients
p   p01 p02                                                      
120 110 110             0          0         0       0          1
        120             0          0         0       0          2


In [ ]:
def event_new_cm_update(new_cm_update):
    